# Define Donor Segment Context &ndash; Total (Giving) Amount Tiered Sample #

## Overview ##

Explore the FEC data by specifying SQL predicates that identify **Donor Segments**, which are static lists of Donor (`donor_indiv` view) records.  Note that a Donor Segment context may including one *or more* segments (e.g. by name or ID).  As with Donor contexts, Donor identities ***are*** discernible within queries using this context type.

For this notebook, we will create Donor Segments containing a sampling of Donors from various tiers of total contribution amounts.  We can then create a Donor Segment context based on one or more of those Donor Segments (cf. the single-segment example in `dc6`).  We use the same simple scheme for grouping Individuals into Donors as described in `dc6`.

This approach will create the following query contexts:

**Principal Context View**

* `ctx_dseg`

**Dependent Context Views**

* `ctx_dseg_memb`
* `ctx_donor`
* `ctx_indiv`
* `ctx_indiv_contrib`
* `ctx_donor_contrib`

## Notebook Setup ##

* Configure database connect information and options
* Clear potentially interfering context (PostgreSQL doesn't let you replace a view definition with conflicting column names)
* Set styling for notebook

In [1]:
sqlconnect = "postgresql+psycopg2://crash@localhost/fecdb"

%load_ext sql
%config SqlMagic.autopandas=True
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%sql $sqlconnect

'Connected: crash@fecdb'

In [2]:
%sql drop view if exists ctx_dseg_memb     cascade
%sql drop view if exists ctx_dseg          cascade
%sql drop view if exists ctx_donor_contrib cascade
%sql drop view if exists ctx_donor         cascade
%sql drop view if exists ctx_household     cascade
%sql drop view if exists ctx_iseg_memb     cascade
%sql drop view if exists ctx_iseg          cascade
%sql drop view if exists ctx_indiv_contrib cascade
%sql drop view if exists ctx_indiv         cascade

 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [3]:
%%html
<style>
  tr, th, td {
    text-align: left !important;
  }
</style>

## Create Donor Segments for Tiered Total Contribution (Sample Donors) ##

Clear out the segment and underlying view(s) used below, if previously created.

For now, let's leave `indiv_group` in place, if existing (i.e. created by `dc6`), since it's kind of expensive to materialize and isn't going to be changing on us at this point (since it is really just a stand-in for more sophistocated logic to come).

In [4]:
%sql delete from donor_seg where name ~ '^\$[0-9].*\+ donors$'
%sql drop table if exists total_amt_seg_def       cascade
%sql drop materialized view if exists donor_sum   cascade
#%sql drop materialized view if exists indiv_group cascade

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


As stated above, we will re-use the current definition/contents of this view, if previously created (see `dc6` for a description of the logic of this view definition).

In [5]:
%%sql
create materialized view if not exists indiv_group as
select ip.part1                  as last_name,
       substr(ip.part2, 1, 3)    as first_name_pfx,
       substr(ip.zip_code, 1, 3) as zip_pfx,
       count(distinct ip.id)     as indivs,
       array_agg(distinct ip.id) as indiv_ids
  from indiv_parsed ip
 where ip.name ~ '^[A-Z][^,]'
   and ip.zip_code is not null
   and ip.num_parts > 1
   and ip.part1 !~ ' '
 group by 1, 2, 3

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


As in `dc6`, we now create a view summarizing the contributions of the Individual groupings (i.e. the approximation of Donors) created above, except we include *all* Individual Contributions here.  The aggregation for each "Donor" includes the list of consolidated `indiv_id` keys, the total number of contributions, and the total and average amounts.

SQL design note: not obvious whether it is better to re-aggregate the unnested ids (even though we are not able to omit the `distinct` qualifier), or select `ig.indiv_ids` and add to GROUP BY clause&mdash;voting for the former option right now.

In [6]:
%%sql
create materialized view donor_sum as
with indiv_group_memb as (
    select ig.last_name,
           ig.first_name_pfx,
           ig.zip_pfx,
           --ig.indiv_ids,
           unnest(ig.indiv_ids) as indiv_id
      from indiv_group ig
)
select igm.last_name,
       igm.first_name_pfx,
       igm.zip_pfx,
       array_agg(distinct igm.indiv_id)
                                 as indiv_ids,
       count(ic.transaction_amt) as contribs,
       sum(ic.transaction_amt)   as total_amt,
       round(sum(ic.transaction_amt) / count(ic.transaction_amt), 2)
                                 as avg_amt
  from indiv_group_memb igm
  join indiv_contrib ic on ic.indiv_id = igm.indiv_id
 group by 1, 2, 3

 * postgresql+psycopg2://crash@localhost/fecdb
6738578 rows affected.


""


Create a couple of indexes for performance.

In [7]:
%sql create index donor_sum_total_amt on donor_sum (total_amt)
%sql create index donor_sum_avg_amt on donor_sum (avg_amt)

 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


We need to record the total number of "Donor" records, total number of contributions, and total contribution amount in `donor_sum` for the cumulative stats below.

In [8]:
%%sql result <<
select count(*)       as num_donors,
       sum(contribs)  as sum_contribs,
       sum(total_amt) as sum_total_amt
  from donor_sum

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.
Returning data to local variable result


In [9]:
(num_donors, sum_contribs, sum_total_amt) = result.loc[0]
# cast from numpy.int64, where needed
num_donors = int(num_donors)
sum_contribs = int(sum_contribs)

61967790

Let's format the results for easier inspection.

In [10]:
%%sql
select to_char(:num_donors,    '999,999,999')        as num_donors,
       to_char(:sum_contribs,  '999,999,999')        as sum_contribs,
       to_char(:sum_total_amt, '999,999,999,999.99') as sum_total_amt

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,num_donors,sum_contribs,sum_total_amt
0,"6,738,578","61,967,790","27,462,347,087.00"


Note that the previous totals only consider the contributions tied to the Individuals included in `donor_sum`.  For perspective, we'll show to percentages of those relative to all contributions in `indiv_contrib`.

In [11]:
%%sql
select round(round(:sum_contribs, 0) / count(*) * 100.0, 2)    as pct_contribs,
       round(:sum_total_amt / sum(transaction_amt) * 100.0, 2) as pct_total_amt
  from indiv_contrib

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,pct_contribs,pct_total_amt
0,97.81,87.50


Enumerate the cutoff amounts (for total contributions) that define the boundaries between the segments to create.

In [12]:
%%sql
create temporary table total_amt_seg_def as
select unnest(array[100000000,
                    50000000,
                    10000000,
                    5000000,
                    1000000,
                    500000,
                    100000,
                    50000,
                    10000,
                    5000,
                    2500,
                    1000,
                    500,
                    250,
                    0]) as seg_amt

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


""


For fun, let's take a look at the number of Donor who have given above each of the threshold amounts we just defined, as well as the percentage of Donors they represent, and a percentage of contribution amount they have given.  Note the percentages are relative to the membership of the `donor_sum` view (and not all contributions), as stated above.

In [13]:
%%sql
select seg_def.seg_amt,
       seg_stat.donors,
       round(seg_stat.donors::numeric / :num_donors * 100, 2) as pct_donors,
       seg_stat.total_amt,
       round(seg_stat.total_amt / :sum_total_amt * 100, 1) as pct_total_amt
  from total_amt_seg_def seg_def
  join lateral
       (select count(*)          as donors,
               sum(ds.total_amt) as total_amt
          from donor_sum ds
         where ds.total_amt > seg_def.seg_amt) as seg_stat on true

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


,seg_amt,donors,pct_donors,total_amt,pct_total_amt
0,100000000,4,0.00,732484158.00,2.7
1,50000000,10,0.00,1140215466.00,4.2
2,10000000,63,0.00,2174445937.00,7.9
3,5000000,152,0.00,2788096190.00,10.2
4,1000000,1272,0.02,4790581969.00,17.4
5,500000,3413,0.05,6239104750.00,22.7
6,100000,27132,0.40,10847056859.00,39.5
7,50000,61836,0.92,13240248957.00,48.2
8,10000,364585,5.41,19338501071.00,70.4
9,5000,713220,10.58,21794567888.00,79.4


Here are a couple functions that will help us create the Donor Segments representing banded giving amounts.

First, a utility function to aid in creating friendlier segments names.

In [14]:
%%sql
CREATE OR REPLACE FUNCTION human_readable(label TEXT) RETURNS TEXT AS $$
    SELECT regexp_replace(
               regexp_replace(
                   regexp_replace(
                       label,
                       '0{9}([^0-9]|$)', 'B\1'),
                   '0{6}([^0-9]|$)', 'M\1'),
               '0{3}([^0-9]|$)', 'K\1');
$$ LANGUAGE SQL

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


Next, the workhorse function that creates a Donor Segment based on a total giving band (i.e. greater than the specified amount, but less than the next higher threshold), and adds appropriate sample Donors as members.  Note that the underlying `create_donor_seg` function does the work of connecting the `indiv_ids` records to the base Donor (`donor_contrib`) record.

Note that this does not really choose a good representative sample within the giving band, it actually just chooses the first `seg_size` records above the threshold amount, and for the case of the top several bands (where the actual membership is less than `seg_size`), it includes members in segments representing higher amounts.  We can fix these things with a little more effort [*psst, EL, let me know*].

In [15]:
%%sql
CREATE OR REPLACE FUNCTION create_donor_seg_by_amt(seg_amt NUMERIC, seg_size INTEGER = 100)
RETURNS TABLE(seg_id BIGINT, seg_name TEXT) AS $$

DECLARE
sql      TEXT;
seg_name TEXT;
seg_desc TEXT = NULL;
BEGIN
    sql = format('select human_readable(concat(%L, $1, %L))', '$', '+ donors');
    EXECUTE sql INTO seg_name USING seg_amt;

    sql = 'with donor_set as (
               select row(ds.indiv_ids)::id_array as ids
                 from donor_sum ds
                where ds.total_amt > $2
                order by ds.total_amt asc
                limit ($3)
           )
           select create_donor_seg(array_agg(ids), $1),
                  $1
             from donor_set
            group by 2';
    RETURN QUERY EXECUTE sql USING seg_name, seg_amt, seg_size;
END;
$$ LANGUAGE plpgsql

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


Now we create Donor Segments for 200 sample records within each segment (with the qualifications on quality of representation given above).  The output lists the segments just created.

In [16]:
%%sql
select seg_def.seg_amt, (create_donor_seg_by_amt(seg_def.seg_amt, 200)).*
  from total_amt_seg_def seg_def

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


,seg_amt,seg_id,seg_name
0,100000000,209,$100M+ donors
1,50000000,210,$50M+ donors
2,10000000,211,$10M+ donors
3,5000000,212,$5M+ donors
4,1000000,213,$1M+ donors
5,500000,214,$500K+ donors
6,100000,215,$100K+ donors
7,50000,216,$50K+ donors
8,10000,217,$10K+ donors
9,5000,218,$5K+ donors


Now show the number of members in each of the segments.  Note that the segments with less then the `seg_size` specified above (and possibly the first one listed with `seg_size` members) may represent multiple inclusions (other than the segment representing the top amount).

In [17]:
%%sql
select ds.id    as seg_id,
       ds.name  as seg_name,
       count(*) as seg_members
  from donor_seg ds
  join donor_seg_memb dsm on dsm.donor_seg_id = ds.id
 where ds.name ~ '^\$[0-9].*\+ donors$'
 group by 1, 2
## Create Principal View (`ctx_dseg`) ## order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
(psycopg2.errors.SyntaxError) syntax error at or near "Create"
LINE 8: ## Create Principal View (`ctx_dseg`) ## order by 1
           ^

[SQL: select ds.id    as seg_id,
       ds.name  as seg_name,
       count(*) as seg_members
  from donor_seg ds
  join donor_seg_memb dsm on dsm.donor_seg_id = ds.id
 where ds.name ~ '^\$[0-9].*\+ donors$'
 group by 1, 2
## Create Principal View (`ctx_dseg`) ## order by 1]
(Background on this error at: http://sqlalche.me/e/f405)


## Create Principal View (`ctx_dseg`) ##

In [18]:
%%sql
create or replace view ctx_dseg as
select id,
       name,
       description
  from donor_seg ds
 where ds.name in ('$100M+ donors', '$50M+ donors', '$10M+ donors')

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


Validate the Segments included in the context.

In [19]:
%%sql
select *
  from ctx_dseg

 * postgresql+psycopg2://crash@localhost/fecdb
3 rows affected.


,id,name,description
0,209,$100M+ donors,None
1,210,$50M+ donors,None
2,211,$10M+ donors,None


## Create Dependent Views ##

### Create `ctx_dseg_memb` ###

In [20]:
%%sql
create or replace view ctx_dseg_memb as
select dsm.*
  from ctx_dseg dsx
  join donor_seg_memb dsm on dsm.donor_seg_id = dsx.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


Taking a look at the Donor Segment Members, we may see some repetition of Donors who were inadvertently added as members of multiple Donor Segments.

In [21]:
%%sql
select ds.id   as dseg_id,
       ds.name as dseg_name,
       d.id    as memb_donor_id,
       d.name  as memb_donor_name,
       d.city,
       d.state,
       d.zip_code
  from ctx_dseg_memb dsmx
  join donor_seg ds on ds.id = dsmx.donor_seg_id
  join donor_indiv d on d.id = dsmx.donor_indiv_id

 * postgresql+psycopg2://crash@localhost/fecdb
77 rows affected.


,dseg_id,dseg_name,memb_donor_id,memb_donor_name,city,state,zip_code
0,209,$100M+ donors,70692,"ADELSON, MIRIAM",LAS VEAGS,NV,89109
1,209,$100M+ donors,70790,"ADELSON, SHELDON",LAS VEGAS,NV,89109
2,209,$100M+ donors,1076630,"BLOOMBERG, MICHAEL",NEW YORK,NY,10021
3,209,$100M+ donors,11612429,"STEYER, THOMAS",SAN FRANCISCO,CA,94104
4,210,$50M+ donors,70692,"ADELSON, MIRIAM",LAS VEAGS,NV,89109
...,...,...,...,...,...,...,...
72,211,$10M+ donors,12215200,"TRUMP, DONALD",NEW YORK,NY,10022
73,211,$10M+ donors,12286548,"UIHLEIN, RICHARD",LAKE BLUFF,IL,60044
74,211,$10M+ donors,12640168,"WARREN, KELCY",DALLAS,TX,75219
75,211,$10M+ donors,13175315,"WOODS, LAURE",MENLO PARK,CA,940287343


### Create `ctx_donor` ###

We actually have to guard againt multiple inclusions of a Donor (member of overlapping Donor Segments), as highlighted by the caveat given above in the creation of the Donor Segments.  If there are a significantly large number of Donor records represented in the context, then we may want to create this as a materialized view, so that the DISTINCT operation isn't performed for every query based on `ctx_donor`.

Note that if there is no chance of multiple inclusion (e.g. if `ctx_dseg` does not include more than one Donor Segments, as is the case in `dc6`), then `ctx_donor` can be created more simply (and more performantly...if that's a word) like this:

```sql
create or replace view ctx_donor as
select d.*
  from ctx_dseg_memb dsmx
  join donor_indiv d on d.id = dsmx.donor_indiv_id
```

In [22]:
%%sql
create or replace view ctx_donor as
with ctx_dseg_donor as (
    select distinct dsmx.donor_indiv_id
      from ctx_dseg_memb dsmx
)
select d.*
  from ctx_dseg_donor dsdx
  join donor_indiv d on d.id = dsdx.donor_indiv_id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


Note that the number of Donor records in the context is less than the number of Donor Segment Members, since the latter count represented multiple inclusions of `donor_indiv` records.

In [23]:
%%sql
select id,
       name,
       city,
       state,
       zip_code,
       elect_cycles
  from ctx_donor

 * postgresql+psycopg2://crash@localhost/fecdb
63 rows affected.


,id,name,city,state,zip_code,elect_cycles
0,5205673,"HENDRICKS, DIANE",AFTON,WI,53501,"[2008, 2010, 2012, 2018]"
1,11306452,"SMITH, THOMAS",SHELOCTA,PA,15774,"[2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]"
2,1726060,"CAMERON, RON",LITTLE ROCK,AZ,72221,[2014]
3,11121369,"SIMONS, JAMES",NEW YORK,NY,10010,"[2010, 2012, 2014, 2016, 2018]"
4,4651340,"GRIFFIN, KEN",CHICAGO,IL,60603,[2020]
...,...,...,...,...,...,...
58,11612429,"STEYER, THOMAS",SAN FRANCISCO,CA,94104,"[2014, 2016, 2018]"
59,9680750,"PRITZKER, JAY",CHICAGO,IL,60606,"[2004, 2012, 2016, 2018]"
60,6756118,"LAMONT, EDWARD",GREENWICH,CT,06813,[2008]
61,11105777,"SIMMONS, HAROLD",DALLAS,TX,75225,"[2004, 2006, 2008, 2010]"


### Create `ctx_indiv` ###

In [24]:
%%sql
create or replace view ctx_indiv as
select i.*
  from ctx_donor dx
  join indiv i on i.donor_indiv_id = dx.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [25]:
%%sql
select id,
       name,
       city,
       state,
       zip_code,
       elect_cycles,
       donor_indiv_id
  from ctx_indiv
 order by donor_indiv_id, name

 * postgresql+psycopg2://crash@localhost/fecdb
1433 rows affected.


,id,name,city,state,zip_code,elect_cycles,donor_indiv_id
0,21801,"ABRAHAM, DANIEL",WEST PALM BEACH,FL,334016161,"[2016, 2018, 2020]",21795
1,21799,"ABRAHAM, DANIEL",WEST PALM BEACH,FL,334016122,"[2014, 2016]",21795
2,21798,"ABRAHAM, DANIEL",WEST PALM BEACH,FL,33401,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014]",21795
3,21802,"ABRAHAM, DANIEL",WEST PALM BEACH,FL,334016163,[2016],21795
4,21800,"ABRAHAM, DANIEL",WEST PALM BEACH,FL,334016152,[2016],21795
...,...,...,...,...,...,...,...
1428,13260839,"YASS, JEFFREY S.",BALA CYNWYD,PA,19004,"[2014, 2016, 2018]",13260803
1429,13260841,"YASS, JEFFREY S.",HAVERFORD,PA,190411804,[2010],13260803
1430,13260840,"YASS, JEFFREY S.",HAVERFORD,PA,19041,[2002],13260803
1431,13260842,"YASS, JEFFREY S MR.",HAVERFORD,PA,19041,[2006],13260803


### Create `ctx_indiv_contrib` ###

In [26]:
%%sql
create or replace view ctx_indiv_contrib as
select ic.*
  from ctx_indiv ix
  join indiv_contrib ic on ic.indiv_id = ix.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [27]:
%%sql
select count(*)             as contribs,
       sum(transaction_amt) as total_amt,
       array_agg(distinct elect_cycle) as elect_cycles
  from ctx_indiv_contrib

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,contribs,total_amt,elect_cycles
0,20567,2174445937.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."


### Create `ctx_donor_contrib` ###

This is really the same as `ctx_indiv_contrib`, except that we are adding `donor_indiv_id` on top of the `indiv_contrib` columns so that queries using this context view are able to join to (and/or group by) the underlying Donor record (and not just the Individual associated with the contribution record).

In [28]:
%%sql
create or replace view ctx_donor_contrib as
select ic.*,
       ix.donor_indiv_id
  from ctx_indiv ix
  join indiv_contrib ic on ic.indiv_id = ix.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [29]:
%%sql
select d.id                 as donor_id,
       d.name               as donor_name,
       count(*)             as contribs,
       sum(transaction_amt) as total_amt,
       array_agg(distinct elect_cycle) as elect_cycles
  from ctx_donor_contrib cx
  join donor_indiv d on d.id = cx.donor_indiv_id
 group by 1, 2
 order by 4 desc, 3 desc

 * postgresql+psycopg2://crash@localhost/fecdb
63 rows affected.


,donor_id,donor_name,contribs,total_amt,elect_cycles
0,11612429,"STEYER, THOMAS",288,259466083.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
1,70790,"ADELSON, SHELDON",521,163010720.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
2,1076630,"BLOOMBERG, MICHAEL",368,162000273.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
3,70692,"ADELSON, MIRIAM",449,148007082.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
4,3535137,"EYCHANER, FRED",602,80924450.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
...,...,...,...,...,...
58,5643646,"HUMPHREYS, DAVID",253,10582540.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
59,21795,"ABRAHAM, DANIEL",89,10379968.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
60,8054934,"MELLON, TIM",101,10273200.00,"[2010, 2012, 2014, 2016, 2018]"
61,948855,"BEZOS, JEFF",7,10159170.00,"[2014, 2016, 2018]"
